## DATA CLEANING & FEATURE SELECTION
## Using Chi-Square test, ANOVA / Kruskal-H Test, Pearson / Spearman Correlation
#### Saneeya Vichare & Jyoti Shree

#### October 23rd

### This involves:
1. Removing all rows belonging to refused or blank categories (7, 9, 99, BLANK, etc.)
2. Removing columns which have more than 30% missing values

### *STEP 1: DATA LOADING*

In [17]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("BRFSS_2024.csv")


/var/folders/1l/dzhvlzcx0k57wvcl_4q0dhlc0000gn/T/ipykernel_4738/2136090270.py:5: DtypeWarning: Columns (288) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("BRFSS_2024.csv")


### *STEP 2: REMOVING BLANK CATEGORIES*

In [18]:
print("=== BEFORE CLEANING ===")
print(f"Original shape: {df.shape}")
initial_missing = df.isnull().sum().sum()
print(f"Initial total NaN values: {initial_missing:,}\n")



# Replace CDC invalid numeric codes with NaN across ALL columns
invalid_codes = [7, 9, 77, 99, 777, 999, 98, 9999]
df = df.replace(invalid_codes, np.nan)

# Replace empty or whitespace-only strings with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Remove rows where DIABETE4 == 2 (gestational diabetes)
if 'DIABETE4' in df.columns:
    before_drop = df.shape[0]
    df = df[df['DIABETE4'] != 2]
    after_drop = df.shape[0]
    print(f"Removed {before_drop - after_drop:,} rows where DIABETE4 == 2")

# Drop rows that have ALL NaN values (but NOT those with just some NaNs)
df_cleaned = df.dropna(how='all')

# Drop columns that are completely empty (100% NaN)
cols_before = set(df.columns)
df_cleaned = df_cleaned.dropna(axis=1, how='all')
cols_after = set(df_cleaned.columns)
dropped_cols = cols_before - cols_after



# Summary of cleaning results
print("=== AFTER CLEANING ===")
print(f"New shape: {df_cleaned.shape}")
final_missing = df_cleaned.isnull().sum().sum()
new_nans = final_missing - initial_missing
print(f"Total NaN values after cleaning: {final_missing:,}")
print(f"New NaN values introduced during cleaning: {new_nans:,}\n")

if dropped_cols:
    print(f"Columns dropped because they were 100% NaN ({len(dropped_cols)} total):")
    print(sorted(list(dropped_cols)))
else:
    print("No columns were fully NaN — none dropped.")

# (Optional) Show top 10 columns with most missing values after cleaning
print("\nTop 10 columns with most missing values after cleaning:\n")
print(df_cleaned.isnull().mean().sort_values(ascending=False).head(10))


=== BEFORE CLEANING ===
Original shape: (457670, 301)
Initial total NaN values: 66,621,255

Removed 3,395 rows where DIABETE4 == 2
=== AFTER CLEANING ===
New shape: (454275, 296)
Total NaN values after cleaning: 66,916,108
New NaN values introduced during cleaning: 294,853

Columns dropped because they were 100% NaN (5 total):
['HPVDSHT', 'ICFQSTVR', 'LCSLAST', 'RCSBORG1', '_AIDTST4']

Top 10 columns with most missing values after cleaning:

COLGHOUS    0.999971
CSRVCTL2    0.998765
CCLGHOUS    0.996872
CSRVINST    0.994605
NOBCUSE8    0.994497
LASTSIG4    0.994486
HPVADSH1    0.993823
CSRVDOC1    0.992927
CSRVSUM     0.992769
CSRVRTRN    0.992399
dtype: float64


### *STEP 3: Remove columns with more than 30% missing values*

In [19]:
# Define threshold
threshold = 0.3  # 30%

# Calculate missing percentage per column
missing_percent = df_cleaned.isnull().mean()

# Identify columns to drop
cols_to_drop = missing_percent[missing_percent > threshold].index.tolist()

# Drop those columns
df_thresholded = df_cleaned.drop(columns=cols_to_drop)

# Print summary
print("=== COLUMN THRESHOLD CLEANING (Step 2) ===")
print(f"Initial columns: {df_cleaned.shape[1]}")
print(f"Columns removed (>30% missing): {len(cols_to_drop)}")
print(f"Remaining columns: {df_thresholded.shape[1]}\n")

if cols_to_drop:
    print("List of removed columns:\n")
    for c in cols_to_drop:
        print("-", c)
else:
    print("No columns exceeded 30% missing — none removed.")

# (Optional) Quick sanity check
print("\nTop 10 columns with most missing values (remaining data):\n")
print(df_thresholded.isnull().mean().sort_values(ascending=False).head(10))


=== COLUMN THRESHOLD CLEANING (Step 2) ===
Initial columns: 296
Columns removed (>30% missing): 177
Remaining columns: 119

List of removed columns:

- CTELENM1
- PVTRESD1
- COLGHOUS
- STATERE1
- CELPHON1
- LADULT1
- NUMADULT
- RESPSLC1
- LANDSEX3
- CCLGHOUS
- POORHLTH
- ASTHNOW
- DIABAGE4
- NUMHHOL4
- NUMPHON4
- EMPLOY1
- INCOME3
- PREGNANT
- HADMAM
- HOWLONG
- CERVSCRN
- CRVCLCNC
- CRVCLPAP
- CRVCLHPV
- HADHYST2
- HADSIGM4
- COLNSIGM
- COLNTES1
- SIGMTES1
- LASTSIG4
- COLNCNCR
- VIRCOLO1
- VCLNTES2
- SMALSTOL
- STOLTEST
- STOOLDN2
- BLDSTFIT
- SDNATES1
- SMOKDAY2
- LCSFIRST
- LCSNUMCG
- LCSSCNCR
- LCSCTWHN
- AVEDRNK4
- DRNK3GE5
- MAXDRNKS
- FLSHTMY3
- IMFVPLA5
- HIVTSTD3
- PDIABTS1
- PREDIAB2
- DIABTYPE
- INSULIN1
- CHKHEMO3
- EYEEXAM1
- DIABEYE1
- DIABEDU1
- FEETSORE
- ARTHEXER
- SHINGLE2
- HPVADVC4
- HPVADSH1
- TETANUS1
- CNCRDIFF
- CNCRAGE
- CNCRTYP2
- CSRVTRT3
- CSRVDOC1
- CSRVSUM
- CSRVRTRN
- CSRVINST
- CSRVINSR
- CSRVDEIN
- CSRVCLIN
- CSRVPAIN
- CSRVCTL2
- PSATEST1
- PSATIME1
-

### *STEP 4: DEFINING VARIABLE TYPES*

In [20]:
# Identify categorical-like numeric columns
categorical_like = [col for col in df_thresholded.columns if df_thresholded[col].nunique() <= 15]

# Everything else is continuous/numeric
continuous_like = [col for col in df_thresholded.columns if df_thresholded[col].nunique() > 15]

print(f"Categorical-like columns: {len(categorical_like)}")
print(f"Continuous-like columns: {len(continuous_like)}")

bmi_cols = ['_BMI5', '_BMI5CAT', '_RFBMI5']

for col in bmi_cols:
    if col in categorical_like:
        print(f"{col}: CATEGORICAL")
    elif col in continuous_like:
        print(f"{col}: CONTINUOUS")
    else:
        print(f"{col}: NOT FOUND in dataset")

Categorical-like columns: 93
Continuous-like columns: 26
_BMI5: CONTINUOUS
_BMI5CAT: CATEGORICAL
_RFBMI5: CATEGORICAL


### *STEP 5: STATISTICAL TESTING*

In [24]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests

# =========================================
# CONFIGURATION
# =========================================
TARGET = 'DIABETE4'
VALID_TARGET_CLASSES = sorted(df[TARGET].dropna().unique())  # e.g., [1, 3, 4]
categorical_cols = categorical_like   # from your earlier step
continuous_cols = continuous_like

# Containers for results
cat_results, cont_results = [], []

# =========================================
# CATEGORICAL FEATURES → Chi-Square test
# =========================================
for col in categorical_cols:
    sub = df[[col, TARGET]].dropna()
    if sub.empty:
        continue
    # Ensure both columns are 1D and contain clean scalar values
    try:
        x = pd.Series(sub[col].astype(str).values.ravel(), name=col)
        y = pd.Series(sub[TARGET].astype(str).values.ravel(), name=TARGET)
        tbl = pd.crosstab(x, y)
    except Exception as e:
        print(f"Skipping {col}: {e}")
        continue

    if tbl.shape[0] < 2 or tbl.shape[1] < 2:
        continue
    try:
        chi2_stat, p, dof, exp = stats.chi2_contingency(tbl, correction=False)
        cat_results.append({
            "feature": col,
            "test": "chi2",
            "statistic": chi2_stat,
            "p_value": p
        })
    except Exception:
        continue

# =======================================================================
# CONTINUOUS FEATURES → ANOVA / Kruskal-H Test + Pearson / Spearman Correlation
# =======================================================================
for col in continuous_cols:
    sub = df[[col, TARGET]].dropna()
    if sub.empty:
        continue

    # ---- ANOVA / Kruskal ----
    groups = [sub.loc[sub[TARGET]==cls, col] for cls in VALID_TARGET_CLASSES]
    try:
        fstat, p = stats.f_oneway(*groups)
        cont_results.append({
            "feature": col,
            "test": "anova",
            "statistic": fstat,
            "p_value": p
        })
    except Exception:
        # fallback: Kruskal-Wallis (non-parametric)
        try:
            stat, p = stats.kruskal(*groups, nan_policy="omit")
            cont_results.append({
                "feature": col,
                "test": "kruskal",
                "statistic": stat,
                "p_value": p
            })
        except Exception:
            pass

    # ---- Pearson / Spearman Correlation ----
    try:
        pearson_r, pearson_p = stats.pearsonr(sub[col], sub[TARGET])
        cont_results.append({
            "feature": col,
            "test": "pearson",
            "statistic": pearson_r,
            "p_value": pearson_p
        })
    except Exception:
        pass

    try:
        spearman_r, spearman_p = stats.spearmanr(sub[col], sub[TARGET])
        cont_results.append({
            "feature": col,
            "test": "spearman",
            "statistic": spearman_r,
            "p_value": spearman_p
        })
    except Exception:
        pass



# =========================================
# COMBINE & ADJUST P-VALUES
# =========================================

results_df = pd.concat([
    pd.DataFrame(cat_results),
    pd.DataFrame(cont_results)
], ignore_index=True).dropna(subset=['p_value'])

# Multiple test correction (Benjamini–Hochberg FDR)
_, p_adj, _, _ = multipletests(results_df['p_value'], method='fdr_bh')
results_df['p_adj'] = p_adj

# Add significance flag
results_df['significant'] = np.where(results_df['p_adj'] < 0.05, 'Yes', 'No')

# Sort by adjusted p-value
results_df = results_df.sort_values('p_adj').reset_index(drop=True)


In [25]:
# =========================================
# DISPLAY RESULTS
# =========================================
print("Top 20 most significant features:")
display(results_df.head(20))

# Optional: summary counts
print("\nSignificant features by test type:")
display(results_df[results_df['significant']=="Yes"]['test'].value_counts())

Top 20 most significant features:


,feature,test,statistic,p_value,p_adj,significant
0,QSTVER,chi2,3742.602755,0.0,0.0,Yes
1,_RACE,chi2,2649.123062,0.0,0.0,Yes
2,_AGE80,pearson,-0.198525,0.0,0.0,Yes
3,_MRACE1,chi2,2047.201241,0.0,0.0,Yes
4,_DRDXAR2,chi2,14092.651698,0.0,0.0,Yes
5,_ASTHMS1,chi2,1779.725113,0.0,0.0,Yes
6,_CASTHM1,chi2,1748.276259,0.0,0.0,Yes
7,_MICHD,chi2,14653.108837,0.0,0.0,Yes
8,_EXTETH3,chi2,14109.452154,0.0,0.0,Yes
9,_RACEGR3,chi2,2233.933442,0.0,0.0,Yes



Significant features by test type:


test
chi2        87
anova       25
pearson     22
spearman    22
Name: count, dtype: int64

### *STEP 6: Feature Importance and extraction*

In [35]:
# FEATURE IMPORTANCE 


# Convert adjusted p-values to -log10 importance
results_df["importance_score"] = -np.log10(results_df["p_adj"].clip(lower=1e-300))

# Normalize to range [0, 1]
results_df["importance_norm"] = (
    results_df["importance_score"] - results_df["importance_score"].min()
) / (results_df["importance_score"].max() - results_df["importance_score"].min())


In [36]:
# SELECTING FEATURES BY THRESHOLD

tau = 0.6   # threshold cutoff
n = 50      # keep at least top 50

selected_by_tau = results_df[results_df["importance_norm"] >= tau]["feature"].tolist()

# Ensure we keep at least top N even if threshold is strict
if len(selected_by_tau) < n:
    selected_by_tau = results_df.head(n)["feature"].tolist()

print(f"Selected {len(selected_by_tau)} features (τ={tau})")
print("Top few:", selected_by_tau[:10])

Selected 96 features (τ=0.6)
Top few: ['QSTVER', '_RACE', '_AGE80', '_MRACE1', '_DRDXAR2', '_ASTHMS1', '_CASTHM1', '_MICHD', '_EXTETH3', '_RACEGR3']


In [37]:
# CRATING CLEAN DATASET WITH THESE NEW FEATURES

TARGET = 'DIABETE4'
df_selected = df[selected_by_tau + [TARGET]].copy()

print("Final shape:", df_selected.shape)


Final shape: (454275, 97)


### *STEP 7: Feature Encoding and Normalization*

In [ ]:
from sklearn.preprocessing import StandardScaler

# One-hot encode categorical columns
X = pd.get_dummies(df_selected.drop(columns=[TARGET]), drop_first=True)
y = df_selected[TARGET]

# Normalize numeric columns
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)